#Create pickle file from a pandas dataframe with targets/labels (annotations) of the helen dataset. 

##Imports.

In [ ]:
import numpy as np
import pandas as pd

import time
from time import sleep
import os
import sys

##Set the path.

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/facial_landmark_pred_example/'
sys.path.append(path)

##Read annotation (label) files.

In [ ]:
def unpack_label(line):
    """
    Get x, y pairs in label file.
    """
    #scale_ratio=0.25

    line = line.replace("\n", "")
    line = line.replace(" ", "")
    split_line = line.split(",")
    
    #x = float(split_line[0]) * scale_ratio
    #y = float(split_line[1]) * scale_ratio
    x = float(split_line[0]) 
    y = float(split_line[1])

    return [x, y]

In [ ]:
def read_label_file(file):    
    """
    Process a single label file.
    Returns the image name & it's corresponding coordinates.
    """
    f = open(file, "r")
    line = f.readline()
    img_name = line.replace("\n", "")
    img_coords = []
    while line:
        line = f.readline()    
        if line != "":   
            img_coords.append(unpack_label(line))
    f.close() 

    return img_name, img_coords

In [ ]:
def read_label_files(path, files):    
    """
    Process all label files.
    Returns all images names & all their corresponding coordinates.
    """  
    imgs_name = []
    imgs_coords = []
    for f in files:
        img_name, img_coords = read_label_file(path + f)
        imgs_name.append(img_name)
        imgs_coords.append(img_coords)
        
    return imgs_name, imgs_coords

###Setup folder path for reading annotation (label) files.

In [ ]:
# setup folder path

data_folder = path + "helen_data/"
print(data_folder)

data_sub_folders = os.listdir(data_folder)
label_path = data_sub_folders[0] + "/" + data_sub_folders[0] + "/"
print(label_path)

# Show files list
files = os.listdir(data_folder + label_path)
print(len(files))
print(files)

###Test read 1 annotation (label) file.

In [ ]:
# Test reading 1 label file

file_name = "2195.txt"      # Just pick a random label file to test.
file = data_folder + label_path + file_name

img_name, img_coords = read_label_file(file)
print(img_name)
print(img_coords)
print(len(img_coords))

###Read all annotation (label) files.

In [ ]:
# Read all label files

import time
start_time = time.time()
imgs_name, imgs_coords = read_label_files(data_folder + label_path, files)
print("--- %s seconds ---" % (time.time() - start_time))

--- 554.7923831939697 seconds ---


In [ ]:
# show a few items in the lists
end=3
print(imgs_name[:end])
print(len(imgs_name[:end]))
print(imgs_coords[:end])
print(len(imgs_coords[:end]))

In [ ]:
# flatten imgs_coords list
imgs_coords_flat = list(np.concatenate(imgs_coords).flat)
#print(imgs_coords_flat)      # IOPub data rate exceeded.

In [ ]:
# reshape imgs_coords_flat
imgs_coords = np.array(imgs_coords_flat).reshape(-1,194*2)     # 194 coordinates
print(imgs_coords.shape)
print(imgs_coords)

###Pack image names & coordinates into a pandas dataframe.

In [ ]:
# Place image file names into dataframe.

imgs_name_df = pd.DataFrame(imgs_name)
imgs_name_df.columns = ['file_name_label']

imgs_name_df

In [ ]:
# Place images coordinates into dataframe.

imgs_coords_df = pd.DataFrame(imgs_coords)
imgs_coords_df

In [ ]:
# Combine the file names & coordinates dataframes.

labels_df = pd.concat([imgs_name_df, imgs_coords_df], axis=1)
labels_df

In [ ]:
"""
Sort the combined dataframe by image file names.
The 1st column is the image file name 
while the rest of the columns are coordinates 
with even columns as Xs & odd columns as Ys for 
all 194 coordinates.
"""

labels_df.sort_values(by=['file_name_label'], inplace=True)
labels_df = labels_df.reset_index(drop=True)
labels_df     # df stores img names & their coordinates.

###Write to pickle file.

In [ ]:
labels_df.to_pickle(path + "helen_pickle/" + "helen_ds_target_pd.pickle")